IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\NO_FAULT1


In [3]:
guasto = 0

VARIABILE RCOU POTENZA MOTORI

In [4]:
#andiamo a prendere una potenza media di 1400 per considerare il momento il cui inizia/finisce il volo
rcou_m1 = pd.read_csv("RCOU.csv")
rcou_m1 = rcou_m1.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1 = rcou_m1[((rcou_m1['C9'] >= potenza) & (rcou_m1['C10'] >= potenza) & (rcou_m1['C11'] >= potenza) & (rcou_m1['C12'] >= potenza) & (rcou_m1['C13'] >= potenza) & (rcou_m1['C14']>= potenza))]

rcou_m1=rcou_m1.reset_index(drop=True)

pd.set_option('display.max_rows',700)
display(rcou_m1)

,TimeUS,C9,C10,C11,C12,C13,C14
0,22664152,1360,1375,1382,1352,1371,1363
1,22676651,1382,1369,1360,1390,1364,1387
2,22771304,1452,1411,1456,1406,1473,1386
3,22782035,1507,1358,1409,1465,1479,1393
4,22792632,1519,1355,1384,1497,1465,1420
...,...,...,...,...,...,...,...
4180,68488846,1553,1460,1514,1503,1422,1584
4181,68499144,1536,1469,1506,1501,1397,1594
4182,68509358,1542,1448,1511,1482,1419,1565
4183,68523070,1572,1386,1463,1509,1415,1550


In [5]:
min=rcou_m1['TimeUS'][0]
max=rcou_m1['TimeUS'][len(rcou_m1)-1]
print(max)
print(min)

68533079
22664152


In [6]:
#xkf utilizzato per fare poi il confronto con i valori di roll, pitch e yaw in att
xkf1_m1 = pd.read_csv("XKF1_0.csv")
xkf1_m1 = xkf1_m1.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1 = xkf1_m1[((xkf1_m1['TimeUS'] >= min) & (xkf1_m1['TimeUS'] <= max))]
xkf1_m1 = xkf1_m1.reset_index(drop=True)
print(xkf1_m1)

        TimeUS  Roll  Pitch    Yaw
0     22675051  0.01  -0.44  13.24
1     22685153  0.03  -0.40  13.24
2     22695762 -0.03  -0.45  13.23
3     22705793 -0.04  -0.52  13.22
4     22715779  0.03  -0.40  13.23
...        ...   ...    ...    ...
4256  68486359 -4.46   1.19   5.63
4257  68496743 -4.40   1.35   5.58
4258  68507280 -4.34   1.52   5.53
4259  68521132 -4.14   1.73   5.49
4260  68531134 -3.95   1.91   5.45

[4261 rows x 4 columns]


In [7]:
att_m1 = pd.read_csv("ATT.csv")
att_m1 = att_m1.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1 = att_m1[((att_m1['TimeUS'] >= min) & (att_m1['TimeUS'] <= max))]
att_m1 = att_m1.reset_index(drop=True)
print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [8]:
for idx,i in enumerate(att_m1['Roll']):
    if(xkf1_m1['Roll'][idx] != i):
        att_m1['Roll'][idx] = ((att_m1['Roll'][idx] + xkf1_m1['Roll'][idx])/2)

for idx,i in enumerate(att_m1['Pitch']):
    if(xkf1_m1['Pitch'][idx] != i):
        att_m1['Pitch'][idx] = ((att_m1['Pitch'][idx] + xkf1_m1['Pitch'][idx])/2)

for idx,i in enumerate(att_m1['Yaw']):
    if(xkf1_m1['Yaw'][idx] != i):
        att_m1['Yaw'][idx] = ((att_m1['Yaw'][idx] + xkf1_m1['Yaw'][idx])/2)

print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1 = pd.read_csv("ESC_0.csv")
esc_0_m1 = esc_0_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1 = esc_0_m1[((esc_0_m1['TimeUS'] >= min) & (esc_0_m1['TimeUS'] <= max))]
esc_0_m1=esc_0_m1.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})
print(esc_0_m1)
esc_1_m1 = pd.read_csv("ESC_1.csv")
esc_1_m1 = esc_1_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1 = esc_1_m1[((esc_1_m1['TimeUS'] >= min) & (esc_1_m1['TimeUS'] <= max))]
esc_1_m1=esc_1_m1.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1 = pd.read_csv("ESC_2.csv")
esc_2_m1 = esc_2_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1 = esc_2_m1[((esc_2_m1['TimeUS'] >= min) & (esc_2_m1['TimeUS'] <= max))]
esc_2_m1=esc_2_m1.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1 = pd.read_csv("ESC_3.csv")
esc_3_m1 = esc_3_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1 = esc_3_m1[((esc_3_m1['TimeUS'] >= min) & (esc_3_m1['TimeUS'] <= max))]
esc_3_m1=esc_3_m1.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1 = pd.read_csv("ESC_4.csv")
esc_4_m1 = esc_4_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1 = esc_4_m1[((esc_4_m1['TimeUS'] >= min) & (esc_4_m1['TimeUS'] <= max))]
esc_4_m1=esc_4_m1.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1 = pd.read_csv("ESC_5.csv")
esc_5_m1 = esc_5_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1 = esc_5_m1[((esc_5_m1['TimeUS'] >= min) & (esc_5_m1['TimeUS'] <= max))]
esc_5_m1=esc_5_m1.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})



       TimeUS  RPM(ESC_0)  Curr(ESC_0)
68   22742780        3495         2.16
69   22845457        4097         2.65
70   22959904        4696         3.34
71   23059012        5180         4.33
72   23167332        5753         4.29
73   23272089        6060         3.92
74   23387634        6310         4.54
75   23499927        6266         4.03
76   23602066        6227         3.98
77   23709666        6214         4.15
78   23821462        6291         4.02
79   23922931        6084         4.91
80   24140073        6394         5.41
81   24241826        6617         4.74
82   24355612        6651         4.40
83   24458193        6428         4.01
84   24563045        6225         3.88
85   24677142        6148         4.24
86   24780143        6059         3.99
87   24894895        6102         3.90
88   25000816        5953         4.13
89   25111147        6179         3.85
90   25216093        6107         3.97
91   25322971        6011         3.83
92   25431084        5965

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1 = pd.read_csv("IMU_0.csv")
imu_0_m1 = imu_0_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1 = imu_0_m1[((imu_0_m1['TimeUS'] >= min) & (imu_0_m1['TimeUS'] <= max))]

imu_1_m1 = pd.read_csv("IMU_1.csv")
imu_1_m1 = imu_1_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1 = imu_1_m1[((imu_1_m1['TimeUS'] >= min) & (imu_1_m1['TimeUS'] <= max))]

imu_2_m1 = pd.read_csv("IMU_2.csv")
imu_2_m1 = imu_2_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1 = imu_2_m1[((imu_2_m1['TimeUS'] >= min) & (imu_2_m1['TimeUS'] <= max))]

imu_m1 = pd.concat((imu_0_m1, imu_1_m1, imu_2_m1))
imu_m1=imu_m1.groupby(imu_m1.TimeUS, as_index=False).mean()

display(imu_m1)


,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,22666080,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081
1,22668252,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686
2,22670635,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578
3,22675772,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946
4,22681012,-0.035033,-0.112585,-0.025552,-0.159212,0.062739,-9.546370
...,...,...,...,...,...,...,...
16503,68522130,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947
16504,68524886,0.331998,0.249299,-0.035411,-0.695731,2.937416,-12.088990
16505,68527023,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677
16506,68530131,0.314659,0.326277,-0.025974,-0.837869,3.015614,-12.519287


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_nofault = pd.merge_ordered(imu_m1,att_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
2      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
3      22675772  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
4      22675940  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
...         ...       ...       ...       ...       ...       ...        ...   
20764  68524886  0.331998  0.249299 -0.035411 -0.695731  2.937416 -12.088990   
20765  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
20766  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
20767  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
20768  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  Pitch  

In [12]:
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_nofault = pd.merge_ordered(m1_nofault, esc_0_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_1_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_2_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_3_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_4_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_5_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
m1_nofault = pd.merge_ordered(m1_nofault, rcou_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")

print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22664152 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3      22668569  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
...         ...       ...       ...       ...       ...       ...        ...   
27405  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27408  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27409  68533079  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  RP

In [13]:
index_to_remove=[]
for ind in att_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m1["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
print("--------------------")
print(len(index_to_remove))

4261
4671
5082
5492
5900
6305
6717
10902
--------------------
10902


In [14]:
#aggiornamento del tempo ponendo a 0 il momento in cui il volo inizia per noi e settaggio della variabile guasto
m1_nofault = m1_nofault[~m1_nofault.TimeUS.isin(index_to_remove)]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"] - m1_nofault.iloc[0]["TimeUS"]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"].astype(int)
m1_nofault["Guasto"] = guasto
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
1             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          2172  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4          4555  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
5          9692  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
8         14932 -0.035033 -0.112585 -0.025552 -0.159212  0.062739  -9.546370   
...         ...       ...       ...       ...       ...       ...        ...   
27401  45856050  0.269569  0.293996 -0.016782 -0.722770  2.779444 -11.869947   
27404  45858806  0.331998  0.249299 -0.035411 -0.695731  2.937416 -12.088990   
27405  45860943  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  45864051  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  45866243  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  Cu

In [15]:
from datetime import timedelta
m1_nofault=m1_nofault.reset_index(drop=True)
m1_nofault['TimeUS'] = pd.to_datetime(m1_nofault['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
1,00:00:00.002172,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
2,00:00:00.004555,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
3,00:00:00.009692,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0
4,00:00:00.014932,-0.035033,-0.112585,-0.025552,-0.159212,0.062739,-9.546370,0.00,0.01,0.00,...,1.90,3348.0,1.99,1382.0,1369.0,1360.0,1390.0,1364.0,1387.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16503,00:00:45.856050,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0
16504,00:00:45.858806,0.331998,0.249299,-0.035411,-0.695731,2.937416,-12.088990,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0
16505,00:00:45.860943,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0
16506,00:00:45.864051,0.314659,0.326277,-0.025974,-0.837869,3.015614,-12.519287,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0


## Analisi nel tempo e in frequenza

In [16]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [17]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_nofault)/350)):
        V1=m1_nofault[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab
    #display(tab)

In [18]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
#result = pd.merge(, how="right")
display(df_merged)


c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_34364\3108494638.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_34364\3108494638.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.034398,0.020502,-0.800395,0.147061,79.0,77.0,4.055489,3.852087,0.019705,...,9052.822458,3.181481,0.524626,-1.163416,3.262661,105.0,103.0,35.821340,7.141273,0
1,1.0,-0.034901,0.020532,-0.814357,0.147283,69.0,67.0,4.243637,3.912545,0.019625,...,8481.195428,3.183892,0.525177,-1.158918,3.265096,105.0,103.0,36.425890,7.748472,0
2,2.0,-0.035228,0.020512,-0.820003,0.147292,81.0,79.0,4.485032,4.010233,0.019617,...,7864.933370,3.186289,0.525713,-1.154421,3.267516,105.0,103.0,37.008907,8.354316,0
3,3.0,-0.035582,0.020498,-0.826986,0.147332,81.0,79.0,4.708145,4.073058,0.019546,...,7206.122551,3.188672,0.526234,-1.149924,3.269920,105.0,103.0,37.570103,8.841604,0
4,4.0,-0.036098,0.020535,-0.840992,0.147580,81.0,79.0,4.865297,4.053457,0.019448,...,6507.270931,3.191042,0.526742,-1.145428,3.272309,107.0,105.0,38.109215,9.211376,0
5,5.0,-0.036543,0.020547,-0.852086,0.147733,73.0,71.0,5.037936,4.033437,0.019447,...,5771.304307,3.193399,0.527235,-1.140931,3.274683,107.0,105.0,38.626006,9.467154,0
6,6.0,-0.036829,0.020519,-0.855587,0.147709,69.0,67.0,5.257739,4.044961,0.019323,...,5001.552185,3.195742,0.527714,-1.136432,3.277042,103.0,101.0,39.120260,9.614721,0
7,7.0,-0.037210,0.020514,-0.863620,0.147786,73.0,71.0,5.434017,3.997716,0.019078,...,4837.286064,3.198073,0.528180,-1.131931,3.279386,107.0,105.0,39.591786,9.661855,0
8,8.0,-0.037719,0.020549,-0.876805,0.148036,75.0,73.0,5.554688,3.880711,0.018976,...,5540.680696,3.200390,0.528633,-1.127428,3.281715,97.0,95.0,40.040417,9.618019,0
9,9.0,-0.038114,0.020548,-0.885318,0.148134,77.0,75.0,5.705865,3.781378,0.018906,...,6219.879554,3.202694,0.529072,-1.122920,3.284030,103.0,101.0,40.466006,9.494044,0


In [19]:
path_file = path_file.replace(r"M1\NO_FAULT1", "")
os.chdir(path_file)
df_merged.to_csv('m1_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
